<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np

In [3]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 28.98 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Checking out files: 100% (2144/2144), done.


In [4]:
df=pd.read_csv("Houses-dataset/Houses Dataset/HousesInfo.txt",header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])

In [8]:
def cleanData(df):
    # compute the number of entries per zipcode
    zipcodes=df['zipcode'].value_counts().keys().tolist()
    counts=df['zipcode'].value_counts().tolist()
    #discard all zipcodes ocurring less than 20 times
    for count,zipcode in zip(counts,zipcodes):
      if count<20:
        idx=df[df['zipcode']==zipcode].index
        df.drop(idx,inplace=True)
    return df

In [9]:
dataset=cleanData(df)

In [10]:
"""
import shutil
from_prefix="Houses-dataset/Houses Dataset/"
to_prefix="images/"
suffix="_frontal.jpg"
!mkdir -p images
for newidx,oldidx in enumerate(df.index.tolist()):
  oldname=from_prefix+str(oldidx)+suffix
  newname=to_prefix+str(newidx)+suffix
  #print("moving from %s to %s"%(oldname,newname))
  shutil.copy(oldname,newname)
  """

In [12]:
#randomize the dataframe
ran_dataset=dataset.sample(len(dataset))

In [21]:
# the dataset has 384 entries. Choose 310 for training and 74 for testing
train_dataset=ran_dataset[0:310]
test_dataset=ran_dataset[310:len(dataset)]
train_dataset.to_csv("train.csv",index_label="index")
test_dataset.to_csv("test.csv",index_label="index")

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,csvFile,imgDir):
    dataset=pd.read_csv(csvFile)
    dummy=pd.get_dummies(dataset['zipcode'])
    price=dataset['price']
    size=dataset['size']
    self.max_size=size.max()
    self.max_price=price.max()
    size=size/self.max_size
    df=dataset.drop(['size','price','zipcode'],axis=1)
    self.data=pd.concat([df,size,dummy,price],axis=1)
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    #
    return self.data.iloc[idx,:-1].to_numpy(dtype=np.float32),np.float32(self.data.iloc[idx,-1]/self.max_price)

In [ ]:
dataset=CustomDataset("cleansedDataset.csv","images")

In [ ]:
dataset.data

,bedrooms,bathrooms,size,91901,92276,92677,92880,93446,93510,94501,94531,price
0,5,3.0,0.264262,0,0,0,0,1,0,0,0,789000
1,3,2.0,0.188968,0,0,0,0,1,0,0,0,365000
2,3,3.0,0.225042,0,0,0,0,1,0,0,0,455000
3,4,2.5,0.258389,1,0,0,0,0,0,0,0,599000
4,2,2.0,0.193477,1,0,0,0,0,0,0,0,529800
...,...,...,...,...,...,...,...,...,...,...,...,...
379,5,2.0,0.216653,0,0,0,0,0,0,0,1,399900
380,4,3.5,1.000000,0,0,0,0,0,0,0,1,460000
381,3,2.0,0.211200,0,0,0,0,0,0,0,1,407000
382,4,3.0,0.242450,0,0,0,0,0,0,0,1,419000


In [ ]:
train_loader=DataLoader(dataset,batch_size=32,shuffle=True)

In [ ]:
itr=iter(train_loader)

In [ ]:
x=next(itr)

In [ ]:
len(x)

2

In [ ]:
x[1].size()

torch.Size([32])

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.relu=nn.ReLU()
    #self.fc1=nn.Linear(in_features=11,out_features=1)
   
    self.fc1=nn.Linear(in_features=11,out_features=32)
    self.fc2=nn.Linear(in_features=32,out_features=16)
    self.fc3=nn.Linear(in_features=16,out_features=1)
  def forward(self,x):
    x=self.fc1(x)
    x=self.relu(x)
    x=self.fc2(x)
    x=self.relu(x)
    x=self.fc3(x)
    return x

In [ ]:
from torch.optim import SGD
from torch.nn import MSELoss


In [ ]:
model=Net()
optimizer=SGD(model.parameters(),lr=0.001)
loss_fn=MSELoss()
epochs=20
for epoch in range(epochs):
  for input,price in train_loader:
    output=model(input)
    loss=loss_fn(output.squeeze(),price)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(loss)
  



tensor(0.0145, grad_fn=<MseLossBackward0>)
tensor(0.0122, grad_fn=<MseLossBackward0>)
tensor(0.0100, grad_fn=<MseLossBackward0>)
tensor(0.0094, grad_fn=<MseLossBackward0>)
tensor(0.0086, grad_fn=<MseLossBackward0>)
tensor(0.0066, grad_fn=<MseLossBackward0>)
tensor(0.0065, grad_fn=<MseLossBackward0>)
tensor(0.0036, grad_fn=<MseLossBackward0>)
tensor(0.0056, grad_fn=<MseLossBackward0>)
tensor(0.0055, grad_fn=<MseLossBackward0>)
tensor(0.0048, grad_fn=<MseLossBackward0>)
tensor(0.0056, grad_fn=<MseLossBackward0>)
tensor(0.0032, grad_fn=<MseLossBackward0>)
tensor(0.0044, grad_fn=<MseLossBackward0>)
tensor(0.0042, grad_fn=<MseLossBackward0>)
tensor(0.0033, grad_fn=<MseLossBackward0>)
tensor(0.0032, grad_fn=<MseLossBackward0>)
tensor(0.0036, grad_fn=<MseLossBackward0>)
tensor(0.0034, grad_fn=<MseLossBackward0>)
tensor(0.0052, grad_fn=<MseLossBackward0>)


In [ ]:
dataset=CustomDataset("cleansedDataset.csv","images")

In [ ]:
itr=iter(dataset)
input,price=next(itr)

In [ ]:
print(input)

[5.         3.         0.26426175 0.         0.         0.
 0.         1.         0.         0.         0.        ]


In [ ]:
max_price=dataset.data['price'].max()

In [ ]:
print(max_price)

5858000


In [ ]:
print(price*max_price)

788999.9755620956


In [ ]:
output=model(torch.tensor(input))

In [ ]:
output.item()*max_price

750707.8570127487

In [ ]:
input,price=next(itr)

In [ ]:
price*max_price

364999.99356269836

In [ ]:
output=model(torch.tensor(input))

In [ ]:
output.item()*max_price

403687.80222535133

In [ ]:
input

array([3.        , 2.        , 0.18896812, 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        ], dtype=float32)